In [1]:
from goodfire import Client
import goodfire
import json
from tqdm import tqdm
import pandas as pd

In [52]:
def query_model(variant,prompt):
    completion=""
    for token in client.chat.completions.create([{"role": "user", "content": prompt}],model=variant,stream=True,max_completion_tokens=150,top_p=0.05):
        completion+=token.choices[0].delta.content
    return completion

def feature_to_json(top_features):
    feature_json = {}
    for feature in top_features:
        feature_json.update({str(feature.feature): feature.activation})
    
    return feature_json

In [3]:
GOODFIRE_API_KEY = 'sk-goodfire-6pFV2uNVzPcn2pmEGUNfhXYVgPo81T89xJrkFJfElNlxFVPmiLGbLA'

In [4]:
with open("few_shot_questions.json","r") as file:
    questions=json.load(file)

In [5]:
client = goodfire.Client(
    GOODFIRE_API_KEY
  )

# Instantiate a model variant
variant = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")

Signature:
client.chat.completions.create(
    messages: Iterable[Union[goodfire.api.chat.interfaces.ChatMessage, dict[str, str]]],
    model: Union[Literal['meta-llama/Meta-Llama-3-8B-Instruct', 'meta-llama/Meta-Llama-3.1-70B-Instruct'], goodfire.variants.variants.VariantInterface],
    stream: bool = False,
    max_completion_tokens: Optional[int] = 2048,
    top_p: float = 0.9,
    temperature: float = 0.6,
    stop: Union[str, list[str], NoneType] = ['<|eot_id|>', '<|begin_of_text|>'],
    timeout: Optional[int] = 320,
    seed: Optional[int] = 42,
    _ChatAPICompletions__system_prompt: str = 'You are a helpful assistant who should follow the users requests. Be brief and to the point, but also be friendly and engaging.',
) -> Union[goodfire.api.chat.interfaces.ChatCompletion, Generator[goodfire.api.chat.interfaces.StreamingChatCompletionChunk, Any, Any]]
Docstring: Create a chat completion.
File:      ~/.pyenv/versions/3.9.6/envs/apart_hackathon/lib/python3.9/site-packages/goodfir

In [99]:
for token in client.chat.completions.create(
    [
        {"role": "user", "content": "Sachin is younger than Rahul by 9 years. If the ratio of their ages is 7 : 9, what is the age of Sachin?"}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=1000
):
    print(token.choices[0].delta.content, end="")

That's a clever question!

Let's break it down step by step. Since the ratio of their ages is 7:9, that means Rahul's age is 9x and Sachin's age is 7x.

We know that Sachin is 9 years younger than Rahul, so we can set up an equation:

9x - 7x = 9

Combine like terms:

2x = 9

Divide both sides by 2:

x = 4.5

So, Rahul's age is 9x = 9(4.5) = 40.5 years old.

And Sachin's age is 7x = 7(4.5) = 31.5 years old.

So, Sachin is 31.5 years old!

In [23]:
def get_top_features(question, topk=10):
    
    with_cot_context = client.features.inspect(
        [
            {
                "role": "user",
                "content": question["cot"]+"\n"+question["question"]
            },
            {
                "role": "assistant",
                "content": question["cot_answer"]
            },
        ],
        model=variant,
    )
    
    without_cot_context = client.features.inspect(
        [
            {
                "role": "user",
                "content": question["direct"]+"\n"+question["question"]
            },
            {
                "role": "assistant",
                "content": question["direct_answer"]
            },
        ],
        model=variant,
    )
    
    with_cot_top_features = with_cot_context.top(k=topk)
    without_cot_top_features = without_cot_context.top(k=topk)
    
    question['with_cot_top_features'] = feature_to_json(with_cot_top_features)
    question['without_cot_top_features'] = feature_to_json(without_cot_top_features)
    
    return question

In [17]:
for question in top_features_list:
    question["with_cot_top_features"] = json.loads(question['with_cot_top_features'])
    question["without_cot_top_features"] = json.loads(question['without_cot_top_features'])

In [10]:
pbar=tqdm(enumerate(questions),total=len(questions))
top_features_list = []
for (k,question) in pbar:
    
    updated_question = get_top_features(question)
    top_features_list.append(updated_question)



100%|██████████| 20/20 [11:46<00:00, 35.33s/it]


In [22]:
with open("top_features_questions.json","w",encoding="utf-8") as file:
     json.dump(top_features_list,file)

In [30]:
without_cot_context = client.features.inspect(
    [
        {
            "role": "user",
            "content": "Q: A juggler can juggle 16 balls. Half of the balls are golf balls, and half of the golf balls are blue. How many blue golf balls are there?\nA: The answer (arabic numerals) is"
        },
        {
            "role": "assistant",
            "content": "8"
        },
    ],
    model=variant,
)

In [31]:
with_cot_context = client.features.inspect(
    [
        {
            "role": "user",
            "content": "Q: A juggler can juggle 16 balls. Half of the balls are golf balls, and half of the golf balls are blue. How many blue golf balls are there?\nA: Let's think step by step"
        },
        {
            "role": "assistant",
            "content": "Sounds like a fun problem! Let's break it down step by step.\nSince the juggler can juggle 16 balls, and half of them are golf balls, that means there are 16/2 = 8 golf balls.\nAnd, half of the golf balls are blue, so we need to find half of 8, which is 8/2 = 4.\nSo, there are 4 blue golf balls!"
        },
    ],
    model=variant,
)

In [75]:
without_cot_context.top(k=10)
feature_to_json(without_cot_context.top(k=10))

In [79]:
json.dumps(feature_to_json(with_cot_context.top(k=10)))

'{"Feature(\\"The model initiating step-by-step explanation\\")": 1.433799342105263, "Feature(\\"Arithmetic operations in word problems\\")": 1.1082763671875, "Feature(\\"Spherical play objects, especially bouncing balls\\")": 1.0290305397727273, "Feature(\\"Types of car transmissions, especially automatic and semi-automatic\\")": 0.8590666118421053, "Feature(\\"Small countable objects\\")": 0.83154296875, "Feature(\\"Numerical values in math problems or quantitative descriptions\\")": 0.8077713815789473, "Feature(\\"Basic Arithmetic and Simple Calculations\\")": 0.7566856971153846, "Feature(\\"Math word problems involving financial calculations\\")": 0.5727796052631579, "Feature(\\"Mathematical word problems involving counting animals or body parts\\")": 0.4872036637931034, "Feature(\\"The model is formulating mathematical equations from word problems or scenarios\\")": 0.4193522135416667}'

In [33]:
with_cot_context.top(k=10)

FeatureActivations(
   0: (Feature("The model initiating step-by-step explanation"), 1.433799342105263)
   1: (Feature("Arithmetic operations in word problems"), 1.1082763671875)
   2: (Feature("Spherical play objects, especially bouncing balls"), 1.0290305397727273)
   3: (Feature("Types of car transmissions, especially automatic and semi-automatic"), 0.8590666118421053)
   4: (Feature("Small countable objects"), 0.83154296875)
   5: (Feature("Numerical values in math problems or quantitative descriptions"), 0.8077713815789473)
   6: (Feature("Basic Arithmetic and Simple Calculations"), 0.7566856971153846)
   7: (Feature("Math word problems involving financial calculations"), 0.5727796052631579)
   8: (Feature("Mathematical word problems involving counting animals or body parts"), 0.4872036637931034)
   9: (Feature("The model is formulating mathematical equations from word problems or scenarios"), 0.4193522135416667)
)

In [29]:
with_cot_context.tokens

[Token("<|begin_of_text|>"),
 Token("<|start_header_id|>"),
 Token("user"),
 Token("<|end_header_id|>"),
 Token("
 
 "),
 Token("A"),
 Token(" store"),
 Token(" sells"),
 Token(" a"),
 Token(" product"),
 Token(" for"),
 Token(" $"),
 Token("60"),
 Token(","),
 Token(" and"),
 Token(" it"),
 Token(" costs"),
 Token(" $"),
 Token("45"),
 Token(" to"),
 Token(" produce"),
 Token("."),
 Token(" If"),
 Token(" the"),
 Token(" store"),
 Token(" sells"),
 Token(" "),
 Token("500"),
 Token(" units"),
 Token(","),
 Token(" what"),
 Token(" is"),
 Token(" the"),
 Token(" total"),
 Token(" profit"),
 Token("?"),
 Token(" Lets"),
 Token(" think"),
 Token(" step"),
 Token(" by"),
 Token(" step"),
 Token("<|eot_id|>"),
 Token("<|start_header_id|>"),
 Token("assistant"),
 Token("<|end_header_id|>"),
 Token("
 
 "),
 Token("Let"),
 Token("'s"),
 Token(" calculate"),
 Token(" the"),
 Token(" total"),
 Token(" profit"),
 Token("!
 "),
 Token("Cost"),
 Token(" to"),
 Token(" produce"),
 Token(" "),
 Tok

In [23]:
print(with_cot_context.tokens[-3])

token_acts = with_cot_context.tokens[-3].inspect()
token_acts

Token("500")


FeatureActivations(
   0: (Feature("Final numerical answers in mathematical calculations"), 2.1875)
   1: (Feature("Large monetary amounts in thousands"), 2.140625)
   2: (Feature("The model is concluding its explanation or calculation"), 1.4296875)
   3: (Feature("Numerical values in text"), 1.2890625)
   4: (Feature("Step-by-step mathematical calculations and reasoning"), 1.09375)
)

### Contrastive Comparison

In [80]:
idx = 19
question = top_features_list[idx]
print("Direct Question:",question["direct"]+"\n"+question["question"])
print("Direct Answer:",question["direct_answer"])

print("Cot Question:",question["cot"]+"\n"+question["question"])

print("Cot Answer:",question["cot_answer"])

print(steering_features_list[idx])


Direct Question: Q: Where do you put your grapes just before checking out? Answer Choices: (A) mouth (B) grocery cart (C) super market (D) fruit basket (E) fruit market
A: The answer is (B).
Q: Google Maps and other highway and street GPS services have replaced what? Answer Choices: (A) united states (B) mexico (C) countryside (D) atlas
A: 
Direct Answer: I think I can help you out!

The correct answer is (D) atlas!
Cot Question: Q: Where do you put your grapes just before checking out? Answer Choices: (A) mouth (B) grocery cart (C) super market (D) fruit basket (E) fruit market
A: The answer should be the place where grocery items are placed before checking out. Of the above choices, grocery cart makes the most sense for holding grocery items. So the answer is (B).
Q: Google Maps and other highway and street GPS services have replaced what? Answer Choices: (A) united states (B) mexico (C) countryside (D) atlas
A: 
Cot Answer: I think I can help you with that!

Google Maps and other hi

In [39]:
def get_steering_features(question):
    
    _, cot_features = client.features.contrast(
    dataset_1=[
        [
            {
                "role": "user",
                "content": question["direct"]+"\n"+question["question"]
            },
            {
                "role": "assistant",
                "content": question["direct_answer"]
            }
        ]
    ],
    dataset_2=[
        
            [ {
                    "role": "user",
                    "content": question["cot"]+"\n"+question["question"]
                },
                {
                    "role": "assistant",
                    "content": question["cot_answer"]
                }]
            ],
        dataset_2_feature_rerank_query="detailed explanation",
        model=variant,
        top_k=5
    )
    
    return cot_features

In [47]:
pbar=tqdm(enumerate(questions),total=len(questions))
steering_features_list = []
for (k,question) in pbar:
    
    top_steering_features = get_steering_features(question)
    steering_features_list.append({"id":k+1,"steering_features":top_steering_features})

100%|██████████| 20/20 [00:31<00:00,  1.59s/it]


### Steering Model according to steering features to check for CoT effect

In [7]:
with open("few_shot_steering_feature_thresholds.json","r") as file:
    steering_thresholds=json.load(file)

In [48]:
test_dataset = pd.read_csv("math_qa_with_answers.csv")
test_dataset_sample = test_dataset.sample(100)

['The model is providing a list of options',
 'Explanatory connectives in step-by-step reasoning',
 'Framing mathematical or logical problem-solving steps',
 'The model is providing scientific classification or technical explanation']

In [62]:
variant.reset()
base_variant = variant
for key in steering_thresholds:
    value = steering_thresholds[key]
    searched_features, relevance = client.features.search(
    key,
    model=variant,
    top_k=1)
    steering_feature = searched_features[0]
    
    variant.reset()
    print(steering_feature,value)
    variant.set(steering_feature, value)
    print(variant)
    
    test_dataset_sample[f'{key}_variant'] = test_dataset_sample['Problem'].apply(lambda x: query_model(variant,x))

Feature("Polite and appreciative language in structured formal communication") 0.5
Variant(
   base_model=meta-llama/Meta-Llama-3-8B-Instruct,
   edits={
      Feature("Polite and appreciative language in structured formal communication"): {'mode': 'nudge', 'value': 0.5},
   }
)
Feature("Data structure and formatting elements") 0.3
Variant(
   base_model=meta-llama/Meta-Llama-3-8B-Instruct,
   edits={
      Feature("Data structure and formatting elements"): {'mode': 'nudge', 'value': 0.3},
   }
)
Feature("Introduction of explanatory statements in question-answering contexts") 0.3
Variant(
   base_model=meta-llama/Meta-Llama-3-8B-Instruct,
   edits={
      Feature("Introduction of explanatory statements in question-answering contexts"): {'mode': 'nudge', 'value': 0.3},
   }
)
Feature("Step-by-step reasoning or problem-solving approach") 0.4
Variant(
   base_model=meta-llama/Meta-Llama-3-8B-Instruct,
   edits={
      Feature("Step-by-step reasoning or problem-solving approach"): {'mode':

In [68]:
test_dataset_sample_part1 = pd.read_csv("test_data_results_part1.csv")
test_dataset_sample_part2 = pd.read_csv("test_data_results_part2.csv")

Index(['The model is providing a list of options_variant',
       'Explanatory connectives in step-by-step reasoning_variant',
       'Framing mathematical or logical problem-solving steps_variant',
       'The model is providing scientific classification or technical explanation_variant',
       'base_variant'],
      dtype='object')

In [76]:
test_data_results = pd.concat([test_dataset_sample_part1,test_dataset_sample_part2[test_dataset_sample_part2.columns[2:]]],axis=1)


Index(['Problem', 'answer',
       'Polite and appreciative language in structured formal communication_variant',
       'Data structure and formatting elements_variant',
       'Introduction of explanatory statements in question-answering contexts_variant',
       'Step-by-step reasoning or problem-solving approach_variant',
       'Action verbs in step-by-step instructions or processes_variant'],
      dtype='object')

In [79]:
base_variant.reset()
print(base_variant)
test_dataset_sample['base_variant_dataset_1'] = test_dataset_sample['Problem'].apply(lambda x: query_model(base_variant,x))

Variant(
   base_model=meta-llama/Meta-Llama-3-8B-Instruct,
   edits={
   }
)


In [82]:
test_dataset_sample[['Problem','answer',"base_variant_dataset_1"]].to_csv("test_dataset1_resuls_BaseVariant.csv", index = False)

In [47]:
query_model(variant,problem)

"A math problem! Let's break it down step by step.\n\nLet's say the two numbers are A and B, and their ratio is 3:4. This means that A is 3/4 of B.\n\nThe LCM (Least Common Multiple) of A and B is 180.\n\nSo, A = 3x and B = 4x, and their LCM is 180.\n\nTo find the values of A and B, we need to find the LCM of 3x and 4x, which is 180.\n\nThe LCM of 3x and 4x is 60x, because 3x and 4x are both multiples of 60.\n\nSo, A = 3x = 60 and B = 4x = 80.\n\nTherefore, the numbers are A = 60 and B = 80.\n\nDid that make sense?"

In [21]:
for token in client.chat.completions.create(
    [
        {"role": "user", "content": "Hola"}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=1000
):
    print(token.choices[0].delta.content, end="")

AttributeError: 'str' object has no attribute 'json'